# ***Portfolio Construction***
We have already calculated the composite score, by giving equal weights to all the factors (i.e. 0.2) and just summing them up.

Now we will continue through the following steps for constructing the portfolio -
*   ***Stock Selection***
*   ***Stock Weighting***
*   ***Monthly Rebalancing***

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
predicted_composite_scores = pd.read_csv('predicted_composite_scores.csv')
volatility_scores = pd.read_csv('volatility_scores.csv')

### ***1) Stock Selection***
I'll be opting out the top-50 stocks on the basis of composite scores and then rebalancing them periodically.

***Periodic Rebalancing*** *- As markets fluctuate, the weights of different assets or stocks in your portfolio naturally shift away from your original allocation. Without rebalancing, you may become unintentionally overexposed to certain assets that have outperformed, increasing your portfolio’s risk beyond your intended tolerance*

In [ ]:
# We have set n=50 i.e., we are selecting the top 50 stocks with periodic rebalancing
# rebalance_freq (str): Rebalancing frequency ('M' for monthly)
def select_top_stocks(composite_scores, top_n=50, rebalance_freq='M'):
    composite_scores['date'] = pd.to_datetime(composite_scores['date'])
    composite_scores['rebalance_date'] = composite_scores['date'] + pd.offsets.MonthEnd(0)

    top_stocks = (
        composite_scores
        .groupby('rebalance_date')
        .apply(lambda x: x.nlargest(top_n, 'composite_score'))
        .reset_index(drop=True)
    )
    return top_stocks[['symbol', 'rebalance_date', 'composite_score']]

### ***2) Stock Weighing***
We'll use two methods for weighing the stocks -
*   ***Equal Weighting -***  All selected stocks have the same weight.
*   ***Risk Adjusted Weighing -*** Consider volatility for weighting. (Inverse of volatility)



In [ ]:
# Assign equal weights to selected stocks
def equal_weighting(selected_stocks):
    df = selected_stocks.copy()
    count = df.groupby('rebalance_date')['symbol'].transform('count')
    df['weight'] = 1 / count
    return df[['symbol', 'rebalance_date', 'weight']]

In [ ]:
# Assign weights based on inverse volatility (risk-adjusted)
def risk_adjusted_weighting(selected_stocks, volatility_scores):
    df = selected_stocks.copy()
    volatility_scores['date'] = pd.to_datetime(volatility_scores['date'])
    df = pd.merge(
        df,
        volatility_scores[['symbol', 'date', 'volatility_score']],
        left_on=['symbol', 'rebalance_date'],
        right_on=['symbol', 'date'],
        how='left'
    )

    # Fill missing volatilities with the mean for each rebalance date
    df['volatility_score'] = df.groupby('rebalance_date')['volatility_score'].transform(
        lambda x: x.fillna(x.mean())
    )

    # If all volatilities are missing for a date, fallback to equal weights
    def assign_weights(group):
        if group['volatility_score'].isnull().all():
            group['weight'] = 1 / len(group)
        else:
            group['inv_vol'] = 1 / (group['volatility_score'] + 1e-8)
            group['weight'] = group['inv_vol'] / group['inv_vol'].sum()
        return group[['symbol', 'rebalance_date', 'weight']]
    df = df.groupby('rebalance_date').apply(assign_weights).reset_index(drop=True)

    return df

### ***Portfolio Construction***

In [ ]:
# Construct portfolio with both weighting methods
def construct_portfolio(composite_scores, volatility_scores, top_n=50):
    top_stocks = select_top_stocks(predicted_composite_scores, top_n)
    equal_weights = equal_weighting(top_stocks)
    risk_weights = risk_adjusted_weighting(top_stocks, volatility_scores)

    return {
        'equal_weight_ml': equal_weights,
        'risk_adjusted_ml': risk_weights
    }

In [ ]:
portfolios = construct_portfolio(composite_scores, volatility_scores)
portfolios['equal_weight_ml'].to_csv('equal_weight_portfolio_ml.csv', index=False)
portfolios['risk_adjusted_ml'].to_csv('risk_adjusted_portfolio_ml.csv', index=False)

/tmp/ipython-input-35-2658524726.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(top_n, 'composite_score'))
/tmp/ipython-input-32-2063693045.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('rebalance_date').apply(assign_weights).reset_index(drop=True)
